In [1]:
from btc_streamer.ml.preprocessing import BTCDataloader
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from xgboost.spark import SparkXGBClassifier
import re
import logging

# Initialize logging
logging.basicConfig(level=logging.INFO)

# Initialize Spark session
spark = SparkSession.builder \
.appName("XGBoostSparkTest") \
.getOrCreate()
logging.info("Spark session initialized successfully.")

# Load and prepare data
btc = BTCDataloader()
btc.setup_spark()
df = btc.load_data(directory='../data/')
df = btc.preproc_split(df)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/25 12:37:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/09/25 12:37:36 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
INFO:root:Spark session initialized successfully.
24/09/25 12:37:40 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/25 12:37:40 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/25 12:37:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/25 12:37:43 WARN WindowExec: No Partition Defined for Window operation! Movin

len dataset: 26579


In [2]:
# Convert feature columns into a single vector column
feature_columns = [x.name for x in df.schema if re.search(r'percent', x.name)]
assembler = VectorAssembler(inputCols=feature_columns, outputCol='features')

assembler

VectorAssembler_c6f9bd947d95

In [3]:
assembled_data = assembler.transform(df).select(['features', 'target'])

In [4]:
train_data, test_data = assembled_data.randomSplit([0.8, 0.2], seed=42)

In [5]:
params = {
        'objective': 'binary:logistic',  # Binary classification
        'booster': 'gbtree',
        'tree_method': 'auto',           # Ensure using CPU
        'num_round': 100,
        'eta': 0.1
    }

In [6]:
!echo $JAVA_HOME

In [7]:
xgb_estimator = SparkXGBClassifier(
        features_col='features',
        label_col='target',use_gpu=False
    )

In [8]:
train_data.show(5)

24/09/25 12:38:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/25 12:38:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/25 12:38:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/25 12:38:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/25 12:38:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/25 12:38:05 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/25 1

+--------------------+------+
|            features|target|
+--------------------+------+
|[-0.1015825605186...|     0|
|[-0.0805882532746...|     0|
|[-0.0750382016173...|     0|
|[-0.0712359669925...|     0|
|[-0.0697370117538...|     0|
+--------------------+------+
only showing top 5 rows



In [9]:
model = xgb_estimator.fit(train_data)

24/09/25 12:38:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/25 12:38:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/25 12:38:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/25 12:38:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/25 12:38:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/25 12:38:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/25 1

Py4JError: An error occurred while calling None.org.apache.spark.api.python.PythonFunction. Trace:
py4j.Py4JException: Constructor org.apache.spark.api.python.PythonFunction([class [B, class java.util.HashMap, class java.util.ArrayList, class java.lang.String, class java.lang.String, class java.util.ArrayList, class org.apache.spark.api.python.PythonAccumulatorV2]) does not exist
	at py4j.reflection.ReflectionEngine.getConstructor(ReflectionEngine.java:180)
	at py4j.reflection.ReflectionEngine.getConstructor(ReflectionEngine.java:197)
	at py4j.Gateway.invoke(Gateway.java:237)
	at py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
	at py4j.commands.ConstructorCommand.execute(ConstructorCommand.java:69)
	at py4j.ClientServerConnection.waitForCommands(ClientServerConnection.java:182)
	at py4j.ClientServerConnection.run(ClientServerConnection.java:106)
	at java.base/java.lang.Thread.run(Thread.java:842)



In [27]:
model

NameError: name 'model' is not defined